In [86]:
from datetime import datetime
import torch
import numpy as np
from pathlib import Path

from utils.settings.config import LINEAR_ENCODER, CROP_ENCODING
num_discrete_labels = len(set(LINEAR_ENCODER.values()))

def starttimer():
    start = datetime.now()
    print(f'{start.strftime("%Y-%m-%d %H:%M:%S")}')
    return start

def endtimer(start):
    end = datetime.now()
    print(f'{end.strftime("%Y-%m-%d %H:%M:%S")}')
    durn = end - start
    print(f'Duration: {durn}')
    return end, durn

In [91]:
run_path = "logs/unet/20220807233511/run_20220807233511/"
run_path = Path(run_path)

testrun_path = Path(run_path / f'testrun_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
testrun_path.mkdir(exist_ok=True, parents=True)

In [8]:
confusion_matrix = torch.zeros([num_discrete_labels, num_discrete_labels])
label = torch.randint(11, (833504,), dtype=torch.uint8, device='cuda')
pred = torch.randint(11, (833504,), dtype=torch.uint8, device='cuda')
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-12 15:19:47
2022-08-12 15:23:54
Duration: 0:04:06.949927


In [9]:
confusion_matrix = torch.zeros([num_discrete_labels, num_discrete_labels])
label = torch.randint(11, (833504,), device='cuda')
pred = torch.randint(11, (833504,), device='cuda')
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-12 15:23:54
2022-08-12 15:27:19
Duration: 0:03:25.371922


In [10]:
confusion_matrix = torch.zeros([num_discrete_labels, num_discrete_labels])
label = torch.randint(11, (833504,), dtype=torch.uint8, device='cpu')
pred = torch.randint(11, (833504,), dtype=torch.uint8, device='cpu')
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-12 15:27:19
2022-08-12 15:28:09
Duration: 0:00:50.333319


In [28]:
confusion_matrix = torch.zeros([num_discrete_labels, num_discrete_labels])
label = torch.randint(11, (833504,), device='cpu')
pred = torch.randint(11, (833504,), device='cpu')
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-12 15:45:05
2022-08-12 15:45:27
Duration: 0:00:22.844359


In [31]:
confusion_matrix = torch.zeros([num_discrete_labels, num_discrete_labels])
confusion_matrix.to(device='cpu')

label = torch.randint(11, (833504,), device='cpu')
pred = torch.randint(11, (833504,), device='cpu')
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-12 15:46:29
2022-08-12 15:46:52
Duration: 0:00:22.485722


In [77]:
from sklearn.metrics import confusion_matrix, classification_report

cm = torch.zeros([num_discrete_labels-1, num_discrete_labels-1])

label = torch.randint(11, (833504,), device='cpu')
pred = torch.randint(11, (833504,), device='cpu')
s = starttimer()
cm_delta = confusion_matrix(label, pred)
cm = cm + cm_delta
e,d = endtimer(s)


2022-08-12 18:56:05
2022-08-12 18:56:05
Duration: 0:00:00.127956


In [78]:
# Calculate and show classification metrics for individual classes
cm = cm.cpu().detach().numpy()
FP = cm.sum(axis=0) - np.diag(cm)
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)


In [81]:
crop_encoding_rev = {v: k for k, v in CROP_ENCODING.items()}
crop_encoding = {k: crop_encoding_rev[k] for k in LINEAR_ENCODER.keys() if k != 0}
#crop_encoding[0] = 'Background/Other'
print(crop_encoding)

class_labels = list(crop_encoding.values())

{110: 'Wheat', 120: 'Maize', 140: 'Sorghum', 150: 'Barley', 160: 'Rye', 170: 'Oats', 330: 'Grapes', 435: 'Rapeseed', 438: 'Sunflower', 510: 'Potatoes', 770: 'Peas'}


In [82]:
cm_report = classification_report(label, pred, zero_division=0, output_dict=True)
print(cm_report)

{'0': {'precision': 0.09121964730865109, 'recall': 0.09136304790086924, 'f1-score': 0.09129129129129128, 'support': 75698}, '1': {'precision': 0.08937364324667761, 'recall': 0.08892166675446452, 'f1-score': 0.08914708212305256, 'support': 75932}, '2': {'precision': 0.08926366669293614, 'recall': 0.09002874667161233, 'f1-score': 0.08964457430039374, 'support': 75487}, '3': {'precision': 0.0911182394236584, 'recall': 0.09175514310678069, 'f1-score': 0.0914355821746128, 'support': 75538}, '4': {'precision': 0.09107557560878508, 'recall': 0.09035613192021592, 'f1-score': 0.09071442733461106, 'support': 75955}, '5': {'precision': 0.0912077039773102, 'recall': 0.0912077039773102, 'f1-score': 0.0912077039773102, 'support': 75805}, '6': {'precision': 0.09059931052771149, 'recall': 0.09007975743194252, 'f1-score': 0.09033878697735911, 'support': 75855}, '7': {'precision': 0.09054600872352725, 'recall': 0.090622205859149, 'f1-score': 0.09058409126754641, 'support': 76052}, '8': {'precision': 0.0

In [85]:
import pandas as pd
# get precision, recall and f1 score
precision = []
recall = []
fscore = []
for cm_class in range(len(class_labels)):
    print(cm_class)
    precision.append(cm_report[str(cm_class)]['precision']*100)
    recall.append(cm_report[str(cm_class)]['recall']*100)
    fscore.append(cm_report[str(cm_class)]['f1-score']*100)

total_accuracy = cm_report['accuracy']

# Convert individual metrics into a dataframe for easier reporting
estimatorMetrics = list(zip(accuracy*100,precision*100,recall*100,fscore*100))

# Add additional rows for summative measures
summative_measures = np.array(['total','weighted avg'])
indices = np.append(class_labels,summative_measures)

# Add Totals
# NOTE: Weighted Average of Recall = Total Accuracy
estimatorMetrics.append([cm_report['accuracy']*100,np.nan,np.nan,np.nan])

# Add Weighted Averages
estimatorMetrics.append([np.nan, cm_report['weighted avg']['precision']*100,
                                 cm_report['weighted avg']['recall']*100,
                                 cm_report['weighted avg']['f1-score']*100])

# Create DataFrame containing metrics
df = pd.DataFrame(estimatorMetrics, index=indices,
                                       columns=['accuracy (%)','precision (%)','recall (%)','fscore (%)'])

display(df.style.format('{0:,.3f}'))

0
1
2
3
4
5
6
7
8
9
10


,accuracy (%),precision (%),recall (%),fscore (%)
Wheat,83.481,9.122,9.136,9.129
Maize,83.446,8.937,8.892,8.915
Sorghum,83.440,8.926,9.003,8.964
Barley,83.474,9.112,9.176,9.144
Rye,83.493,9.108,9.036,9.071
Oats,83.470,9.121,9.121,9.121
Grapes,83.490,9.060,9.008,9.034
Rapeseed,83.397,9.055,9.062,9.058
Sunflower,83.429,9.254,9.253,9.253
Potatoes,83.467,9.072,9.117,9.094
